In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)
print(tf.test.is_built_with_cuda())


Num GPUs Available:  0
2.17.0
False


In [3]:
# Load Food101 dataset
food101_train, food101_test = tfds.load('food101', split=['train', 'validation'], as_supervised=True)

In [4]:
# Load Fruits-360 dataset
fruits360_train = image_dataset_from_directory(
    'fruits-360_dataset_100x100/fruits-360/Training',
    image_size=(224, 224),
    batch_size=None,
    label_mode='int'
)

fruits360_test = image_dataset_from_directory(
    'fruits-360_dataset_100x100/fruits-360/Test', 
    image_size=(224, 224),
    batch_size=None,
    label_mode='int'
)

Found 70491 files belonging to 141 classes.
Found 23619 files belonging to 141 classes.


In [5]:
# Define a function to preprocess images and labels
def preprocess_data(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (224, 224))
    label = tf.cast(label, tf.int64)  # Ensure all labels are int64
    return image, label

# Apply preprocessing to all datasets
food101_train = food101_train.map(preprocess_data)
food101_test = food101_test.map(preprocess_data)
fruits360_train = fruits360_train.map(preprocess_data)
fruits360_test = fruits360_test.map(preprocess_data)


In [6]:
# Combine the datasets
train_dataset = food101_train.concatenate(fruits360_train)
test_dataset = food101_test.concatenate(fruits360_test)

In [7]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [8]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

In [9]:
# Load a pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [10]:
# Fine-tuning: Unfreeze the top layers of the base model
base_model.trainable = True
fine_tune_at = 100  

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [11]:
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(242, activation='softmax')  # Adjust the number of classes if needed
])

C:\Users\Troxi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [12]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2736s 597ms/step - accuracy: 0.4337 - loss: 2.4928 - val_accuracy: 0.5502 - val_loss: 2.2864
Epoch 2/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2681s 586ms/step - accuracy: 0.6233 - loss: 1.4839 - val_accuracy: 0.6930 - val_loss: 1.2092
Epoch 3/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2770s 606ms/step - accuracy: 0.6850 - loss: 1.2033 - val_accuracy: 0.7478 - val_loss: 0.9777
Epoch 4/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2652s 580ms/step - accuracy: 0.7183 - loss: 1.0528 - val_accuracy: 0.7646 - val_loss: 0.9109
Epoch 5/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2708s 592ms/step - accuracy: 0.7485 - loss: 0.9384 - val_accuracy: 0.7799 - val_loss: 0.8416
Epoch 6/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2767s 605ms/step - accuracy: 0.7675 - loss: 0.8512 - val_accuracy: 0.7909 - val_loss: 0.7992
Epoch 7/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2995s 655ms/step - accuracy: 0.7869 - loss: 0.7758 - val_accuracy: 0.7978 - val_loss: 0.7675
Epoch 8/10
4571/4571 ━━━━━━━━━━━━━━━━━━━━ 2969s 649ms/s

In [14]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('food_model_combined_v2.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Troxi\AppData\Local\Temp\tmpk_w87_gv\assets


INFO:tensorflow:Assets written to: C:\Users\Troxi\AppData\Local\Temp\tmpk_w87_gv\assets


Saved artifact at 'C:\Users\Troxi\AppData\Local\Temp\tmpk_w87_gv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 242), dtype=tf.float32, name=None)
Captures:
  1681410905040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410909840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410904080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410904848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410908688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410910224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410910800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410909456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410911760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1681410911376: TensorSpec(shape=(), dtype=tf.resource, name=None)
 